In [43]:
from insolAPI.WebAPI import API
import simplejson as json
import pandas as pd
import pendulum as pdl
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from datetime import datetime, timedelta
import warnings
from tqdm import tqdm
import sys
from tabulate import tabulate
from pytz import timezone
import numpy as np

In [50]:
time_args = dict(
    start=pdl.yesterday().subtract(weeks=1).to_datetime_string(),
    stop=(pdl.now()).to_datetime_string(),
    timezone = timezone('Europe/Zurich')
)

def read_json_config():
    """
    Read the config.json file and return the api key
    """
    with open("config.json") as f:
        config_data = json.load(f)

    installations = {}

    # Iterate through locations
    locations = config_data['locations']
    for location in locations:
        # Store details in a dictionary
        location_details = {
            'id': location['id'],
            'name': location['name'],
            'latitude': location['latitude'],
            'longitude': location['longitude'],
            'wind_threshold': location['wind_threshold'],
            'high_temperature_threshold': location['high_temperature_threshold'],
            'has_a_screen': location['has_a_screen'],
        }
        installations[location['name']] = location_details
    return installations, [config_data['api_key'], config_data['api_url']]


installation_path = "C:/Users/Insolight/Desktop/InsolReports/Scripts"
with open(installation_path + "/local.json") as f:
    local_data = json.load(f)
api = API(local_data["API_user"], local_data["API_pwd"], dev_space=False)

list_sensor = [
    "PAR",
    "IRRAD",
    "GII",
    "DNI",
    "DHI",
    "TEMP",
    "HUMI",
    "RAIN",
    "RAIN_TYPE",
    "RAIN_ACCUMULATED",
    "WIND",
    "WIND_DIR",
    "VIRTUAL",
    "LEAF_TEMP"
]

dict_instal = {
    "Etchelecu": 26,
    "Agroscope Series": 24,
    "Agroscope Berries": 25,
    "Bioschmid": 23,
}

def print_progress_bar(percentage, length=10):
    # print(percentage)
    if np.isnan(percentage):
        percentage = 1
    block = int(round(length * percentage))
    progress = "[" + "#" * block + "-" * (length - block) + "]"
    # print(f"\r{progress}", end="", flush=True)
    return progress

In [3]:
logs_joined = {}
dict_list_theoretical = {}
for instal in tqdm(dict_instal):
    logs_joined[instal] = {}
    dict_list_theoretical[instal] = []
    sensor_number = 0
    for sensor_type in api.SensorsTypes:
        if str(sensor_type).split(".")[1] in list_sensor:
            with warnings.catch_warnings():
                warnings.simplefilter("ignore", category=UserWarning)
                logs_joined[instal][list_sensor[sensor_number]] = api.get_sensor_channels_logs_joined(**time_args,sensor_type=sensor_type, install=dict_instal[instal])
                sensor_number += 1
                try:
                    theoretical_sensor = api.get_sensor_channels(sensor_type=sensor_type, install=dict_instal[instal])
                    theoretical_sensor = theoretical_sensor[theoretical_sensor["deleted_at"].isna()].sensor_name.unique()
                    dict_list_theoretical[instal].extend(theoretical_sensor)
                except:
                    pass

100%|██████████| 4/4 [00:42<00:00, 10.51s/it]


In [51]:
unique_sensors = {}
logs_joined_unique = {}
last_log = {}
time_diff = {}
for instal in dict_instal:
    unique_sensors[instal] = []
    time_diff[instal] = {}
    for sensor_type in logs_joined[instal]:
        try:
            unique_sensors[instal].extend(logs_joined[instal][sensor_type]["sensor_name"].unique())
        except :
            pass
    unique_sensors[instal] = list(set(unique_sensors[instal]))
    logs_joined_unique[instal] = {}
    for sensor_type in logs_joined[instal]:
        for unique_sensor in unique_sensors[instal]:
            try:
                if unique_sensor in logs_joined[instal][sensor_type]["sensor_name"].unique():
                    logs_joined_unique[instal][unique_sensor] = logs_joined[instal][sensor_type].loc[logs_joined[instal][sensor_type]["sensor_name"] == unique_sensor]
                    logs_joined_unique[instal][unique_sensor].index = logs_joined_unique[instal][unique_sensor].index.round('min')
                    logs_joined_unique[instal][unique_sensor] = logs_joined_unique[instal][unique_sensor].loc[~logs_joined_unique[instal][unique_sensor].index.duplicated(keep='first')]
            except :
                pass


    for sensor in logs_joined_unique[instal]:
        logs_joined_unique[instal][sensor] = logs_joined_unique[instal][sensor].dropna(subset=[logs_joined_unique[instal][sensor].columns[1]])
        time_difference = logs_joined_unique[instal][sensor].index.to_series().diff()
        time_difference = time_difference[time_difference > pd.Timedelta(minutes=2)].sum()
        time_diff[instal][sensor] = time_difference

    last_log[instal] = {}
    for sensor in dict_list_theoretical[instal]:
        try:
            last_log[instal][sensor] = logs_joined_unique[instal][sensor].index[-1]
        except:
            last_log[instal][sensor] = "> 1 week"
        try:
            if last_log[instal][sensor] > pdl.now().subtract(minutes=10):
                last_log[instal][sensor] = "Online"
        except:
            pass

#make a df with the last log and the time difference for each sensor
dict_df = {}
for instal in last_log:
    dict_df[instal] = pd.DataFrame.from_dict(last_log[instal], orient="index", columns=["Last log"])
    dict_df[instal]["Time offline (1w)"] = dict_df[instal].index.map(time_diff[instal])

    dict_df[instal]["Progress"] = dict_df[instal]["Time offline (1w)"].apply(lambda x: print_progress_bar(x.total_seconds() / (7 * 24 * 60 * 60)))

# #sort the sensors by the last log
# for instal in last_log:
#     last_log[instal] = {k: v for k, v in sorted(last_log[instal].items(), key=lambda item: item[1])}


for instal, df in dict_df.items():
    # Sort the DataFrame by the "Last log" column
    df_sorted = df.sort_values(by=["Last log", "Time offline (1w)"], ascending=[True, False])

    print(f"{instal}")
    table = tabulate(df_sorted, headers="keys", tablefmt="psql", showindex=True)
    print(table)
    print("\n")

#add a column to the df with the progress bar, knowing that 100% is 1 week



# saving = input("Save table to excel file? (y/n)")
# if saving == "y":
#     writer = pd.ExcelWriter('reports/last_log.xlsx', engine='openpyxl')
#     for instal, df in dict_df.items():
#         df.to_excel(writer, sheet_name=instal)
#     writer.close()
#     print("File saved as 'last_log.xlsx'")


0.05327380952380952
0.05327380952380952
0.05376984126984127
0.05376984126984127
0.05376984126984127
0.05376984126984127
0.05376984126984127
0.05376984126984127
0.05357142857142857
0.05367063492063492
0.05347222222222222
0.05396825396825397
0.05376984126984127
0.05992063492063492
0.05426587301587302
0.05357142857142857
0.05357142857142857
0.05357142857142857
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.00029761904761904765
0.001488095238095238
0.0008928571428571428
0.0008928571428571428
0.0008928571428571428
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
nan
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
nan
0.00029761904761904765
0.00029761904761904765
0.00029761904761904765
0.00029761904761904765
0.00029761904761904765
0.00029761904761904765
0.00029761904761904765
0.00029761904761904765
0.00029761904761904765
0.00029761904761904765
0.00029761904761904765
0.00029761904761904765
0.00029761904761904765
0.00029761904761904765
0.000297

In [45]:
rows_num = len(logs_joined_unique["Etchelecu"]["TRH_bottom_EXT"])
#substract the last index to the first index to get the time difference
elapsed_time = logs_joined_unique["Etchelecu"]["TRH_bottom_EXT"].index[-1] - logs_joined_unique["Etchelecu"]["TRH_bottom_EXT"].index[0]
(elapsed_time.total_seconds()/60 - rows_num)

577.0

In [41]:
logs_joined_unique["Etchelecu"]["TRH_bottom_EXT"].index.duplicated(keep='first').sum()

0

In [50]:
logs_joined_unique["Etchelecu"]["TRH_bottom_EXT"].index[-1]

Timestamp('2023-12-21 09:32:00+0100', tz='Europe/Zurich')

In [51]:
time_diff = timedelta()
for ind, row in logs_joined_unique["Etchelecu"]["TRH_bottom_EXT"].iterrows():
    if ind < logs_joined_unique["Etchelecu"]["TRH_bottom_EXT"].index[-1]:
        if ind + timedelta(minutes=1) != logs_joined_unique["Etchelecu"]["TRH_bottom_EXT"].index[logs_joined_unique["Etchelecu"]["TRH_bottom_EXT"].index.get_loc(ind) + 1]:
            # print(ind, logs_joined_unique["Etchelecu"]["TRH_bottom_EXT"].index[logs_joined_unique["Etchelecu"]["WIND01_EXT"].index.get_loc(ind) + 1])
            #compute the time difference between the two rows
            time_diff += logs_joined_unique["Etchelecu"]["TRH_bottom_EXT"].index[logs_joined_unique["Etchelecu"]["TRH_bottom_EXT"].index.get_loc(ind) + 1] - ind

In [52]:
time_diff

Timedelta('0 days 11:53:00')

In [66]:
time_diff_2 = logs_joined_unique["Etchelecu"]["TRH_bottom_Insolagrin"].index.to_series().diff()
time_no_logs = time_diff_2[time_diff_2 > pd.Timedelta(minutes=1)].sum()
print(f"Number of times elapsed with no logs: {time_no_logs}")

Number of times elapsed with no logs: 3 days 13:29:00


In [64]:
for i in logs_joined_unique:
    print("\n")
    print(i)
    for j in logs_joined_unique[i]:
        print(j)



Etchelecu
PAR02_control
PAR03_Insolagrin
PAR01_EXT
PAR04_Insolagrin
PAR01_Insolagrin
PAR02_EXT
PAR01_control
PAR02_Insolagrin
GHI_01_EXT
TRH_top_Insolagrin
TRH_bottom_Insolagrin
TRH_top_control
TRH_top_EXT
TRH_bottom_EXT
TRH_bottom_control
RAIN_EXT
WIND02_EXT
WIND01_EXT


Agroscope Series
PAR4
PAR_STRAWB
PAR_EXT_06
PAR_EXT_05
PAR8
PAR_umbrella_strawb
PAR3
PAR2
PAR1
PAR7
PAR_EXT_04
PAR01_Nursery
PAR5
PAR_ext_bis
PAR_umbrella
PAR6
PAR01_TX_tunnel
PAR01_GT_tunnel
PAR01_GT1_tunnel
GHI_EXT_03
GHI_EXT_02
weather_station
GHI_EXT_04
GHI_EXT
TRH01_TX_tunnel
TRH_top_Nursery
TRH02_GT_tunnel
TRH_bottom_Nursery
T_RH1
TRH01_GT1_tunnel
T_RH_umbrella_bottom
T_RH_umbrella_top
T_RH_ext_bottom
TRH01_GT_tunnel
T_RH2
TRH_EXT_top
TRH02_GT1_tunnel
wind_speed
wind_speed_ultrasonic
leaf_temp_umbrella1
LeafTemp_Vajolet_W_NurseryTunnel
LeafTemp_Vajolet_E_NurseryTunnel
LeafTemp01_Flair_Insolagrin05_TX
LeafTemp02_Flair_Insolagrin05_TX
LeafTemp_Tulameen_E_NurseryTunnel
LeafTemp01_Joly_Insolagrin05_TX
LeafTemp_Vaj

# Plotting

# Plot the distribution of the time between logs

In [51]:
time_diff = {}
for instal in dict_instal:
    time_diff[instal] = {}
    for unique_sensor in logs_joined_unique[instal]:
        #compute the time difference between two logs in seconds
        time_diff[instal][unique_sensor] = logs_joined_unique[instal][unique_sensor].index.to_series().diff()[1:]

In [ ]:
for instal in dict_instal:
    total_num_sensors = len(unique_sensors[instal])
    num_rows = total_num_sensors // 6
    fig = plt.figure()
    fig.set_figheight(7*num_rows)
    fig.set_figwidth(20)
    fig.suptitle(instal)
    for unique_sensor in unique_sensors[instal]:
        ax = fig.add_subplot(num_rows + 1, 6, unique_sensors[instal].index(unique_sensor) + 1)
        ax.hist(time_diff[instal][unique_sensor].dropna().dt.total_seconds(), bins=10, range=(61, 10000))
        ax.set_title(unique_sensor)
        ax.set_xlim(61, 10000)
        ax.grid(True)
        ax.set_yscale('log')
    fig.tight_layout()

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

for instal in dict_instal:
    total_num_sensors = len(unique_sensors[instal])
    num_rows = total_num_sensors // 6

    fig = make_subplots(rows=num_rows + 1, cols=6, subplot_titles=unique_sensors[instal], shared_yaxes=True)
    fig.update_layout(title_text=instal)
    #set the height of the figure
    fig.update_layout(height=300*num_rows)

    for i, unique_sensor in enumerate(unique_sensors[instal]):
        row_num = i // 6 + 1
        col_num = i % 6 + 1

        trace = go.Histogram(
            x=time_diff[instal][unique_sensor].dropna().dt.total_seconds(),
            nbinsx=10,
            xbins=dict(start=61, end=10000),
            name=unique_sensor,
        )

        fig.add_trace(trace, row=row_num, col=col_num)

        fig.update_xaxes(row=row_num, col=col_num)
        fig.update_yaxes(row=row_num, col=col_num, type="log")

    fig.update_layout(showlegend=False)
    # fig.update_layout(subplot_titles_text={'size': 10})
    fig.show()
    #save the figure as a html file
    fig.write_html("hist log frequency/" + instal + "_time_diff.html")


# TESTS

In [35]:
test = api.get_sensor_channels(sensor_type=api.SensorsTypes.LEAF_TEMP, install=24)

In [36]:
a = test[test["deleted_at"].isna()]
a

,gateway_id,local_channel_id,created_at,deleted_at,sensor_name,channel_name,sensor_type,position_bitfield,brand,model,serial_number,configuration_json,address,dx,dy,dz,nx,ny,zones_ids,zones_names
id,,,,,,,,,,,,,,,,,,,,
995,29,769,2023-08-25T10:28:50.000000Z,None,LeafTemp_Tulameen_E_NurseryTunnel,temp_sensor,234,22,Calex,InfraredSensor,unknown,[],136,0,0,0,1,1,[27],[Nursery Tunnel]
994,29,768,2023-08-25T10:28:50.000000Z,None,LeafTemp_Tulameen_E_NurseryTunnel,temp_raw,234,22,Calex,InfraredSensor,unknown,[],136,0,0,0,1,1,[27],[Nursery Tunnel]
993,29,767,2023-08-25T10:28:50.000000Z,None,LeafTemp_Tulameen_E_NurseryTunnel,temp_max,234,22,Calex,InfraredSensor,unknown,[],136,0,0,0,1,1,[27],[Nursery Tunnel]
992,29,766,2023-08-25T10:28:50.000000Z,None,LeafTemp_Tulameen_E_NurseryTunnel,temp_min,234,22,Calex,InfraredSensor,unknown,[],136,0,0,0,1,1,[27],[Nursery Tunnel]
991,29,765,2023-08-25T10:28:50.000000Z,None,LeafTemp_Tulameen_E_NurseryTunnel,temp_avg,234,22,Calex,InfraredSensor,unknown,[],136,0,0,0,1,1,[27],[Nursery Tunnel]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
770,29,669,2023-07-29T00:24:19.000000Z,None,leaf_temp_FS1,temp_sensor,234,22,Calex,InfraredSensor,OL091594,[],130,0,0,0,1,1,[],[]
769,29,668,2023-07-29T00:24:19.000000Z,None,leaf_temp_FS1,temp_raw,234,22,Calex,InfraredSensor,OL091594,[],130,0,0,0,1,1,[],[]
768,29,667,2023-07-29T00:24:19.000000Z,None,leaf_temp_FS1,temp_max,234,22,Calex,InfraredSensor,OL091594,[],130,0,0,0,1,1,[],[]


In [17]:
logs_joined_unique["Etchelecu"]["TRH_top_Insolagrin"]

,sensor_channel_id,Humidity [%],integrated [% · h],gateway_id,local_channel_id,created_at,deleted_at,sensor_name,channel_name,sensor_type,...,serial_number,configuration_json,address,dx,dy,dz,nx,ny,zones_ids,zones_names
2023-12-14 00:01:00+01:00,1113,99.9,1.193250,31,1,2023-11-09T10:48:31.000000Z,None,TRH_top_Insolagrin,humidity,252,...,unknown,[],43,0,0,0,1,1,[39],[Insolagrin]
2023-12-14 00:02:00+01:00,1113,99.9,2.858250,31,1,2023-11-09T10:48:31.000000Z,None,TRH_top_Insolagrin,humidity,252,...,unknown,[],43,0,0,0,1,1,[39],[Insolagrin]
2023-12-14 00:04:00+01:00,1113,99.9,6.188250,31,1,2023-11-09T10:48:31.000000Z,None,TRH_top_Insolagrin,humidity,252,...,unknown,[],43,0,0,0,1,1,[39],[Insolagrin]
2023-12-14 00:05:00+01:00,1113,99.9,7.853250,31,1,2023-11-09T10:48:31.000000Z,None,TRH_top_Insolagrin,humidity,252,...,unknown,[],43,0,0,0,1,1,[39],[Insolagrin]
2023-12-14 00:06:00+01:00,1113,99.9,9.518250,31,1,2023-11-09T10:48:31.000000Z,None,TRH_top_Insolagrin,humidity,252,...,unknown,[],43,0,0,0,1,1,[39],[Insolagrin]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-12-22 09:20:00+01:00,1113,99.9,931.928223,31,1,2023-11-09T10:48:31.000000Z,None,TRH_top_Insolagrin,humidity,252,...,unknown,[],43,0,0,0,1,1,[39],[Insolagrin]
2023-12-22 09:21:00+01:00,1113,99.9,933.593262,31,1,2023-11-09T10:48:31.000000Z,None,TRH_top_Insolagrin,humidity,252,...,unknown,[],43,0,0,0,1,1,[39],[Insolagrin]
2023-12-22 09:22:00+01:00,1113,99.9,935.258240,31,1,2023-11-09T10:48:31.000000Z,None,TRH_top_Insolagrin,humidity,252,...,unknown,[],43,0,0,0,1,1,[39],[Insolagrin]
2023-12-22 09:24:00+01:00,1113,99.9,938.588257,31,1,2023-11-09T10:48:31.000000Z,None,TRH_top_Insolagrin,humidity,252,...,unknown,[],43,0,0,0,1,1,[39],[Insolagrin]


In [30]:
time_delta = logs_joined_unique["Etchelecu"]["TRH_top_Insolagrin"].index[-1] - logs_joined_unique["Etchelecu"]["TRH_top_Insolagrin"].index[0]
#convert time delta to minutes
time_delta_min = time_delta.total_seconds()/60
#compute the number of rows
rows_num = len(logs_joined_unique["Etchelecu"]["TRH_top_Insolagrin"])
#compute the number of rows that are missing
missing_rows = time_delta_min - rows_num
missing_rows
#count the number of times that there is no log for more than 1 minute
time_diff = logs_joined_unique["Etchelecu"]["TRH_top_Insolagrin"].index.to_series().diff()
time_no_logs = time_diff[time_diff > pd.Timedelta(minutes=2)].sum()
# print(f"Number of times elapsed with no logs: {time_no_logs}")
time_no_logs

Timedelta('0 days 09:07:00')

In [15]:
#same with plotly
fig = go.Figure()
fig.add_trace(go.Scatter(x=logs_joined_unique["Etchelecu"]["TRH_top_Insolagrin"].index, y=logs_joined_unique["Etchelecu"]["TRH_top_Insolagrin"]["Humidity [%]"]))
fig.show()


In [37]:
import time
def print_progress_bar(percentage, length=10):
    block = int(round(length * percentage))
    progress = "[" + "#" * block + "-" * (length - block) + "]"
    print(f"\r{progress}", end="", flush=True)

# Example usage:
for i in range(101):
    percentage = i / 100.0
    print_progress_bar(percentage)
    time.sleep(0.1)  # Simulate some computation


[##--------]

KeyboardInterrupt: 

In [38]:
print_progress_bar(0.5)

[#####-----]